In [ ]:
import torch
from torch import nn

# Dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

# MLP simples em PyTorch (1 camada oculta)
def make_mlp(n_in, n_hidden, n_out=1, dropout=0.0):
    return nn.Sequential(
        nn.Linear(n_in, n_hidden),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(n_hidden, n_out)  # saída = logit (sem sigmoid aqui)
    ).to(device)

# Função de treino minimalista
def train(X, y, n_hidden=8, lr=0.1, epochs=100, verbose=False):
    # Tensores
    X_t = torch.tensor(X, dtype=torch.float32, device=device)
    y_t = torch.tensor(y.reshape(-1,1), dtype=torch.float32, device=device)

    model = make_mlp(X_t.shape[1], n_hidden, n_out=1)

    criterion = nn.BCEWithLogitsLoss()             # Loss binária
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)  # SGD puro

    losses = []
    for ep in range(epochs):
        optimizer.zero_grad()
        logits = model(X_t)
        loss = criterion(logits, y_t)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        if verbose and (ep % max(1, epochs//10) == 0):
            print(f"época {ep:4d} | loss={loss.item():.6f}")

    return model, losses